In [4]:
pip install wordcloud

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 168.4/168.4 kB 4.6 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [5]:
# import important modules
import numpy as np
import pandas as pd

import re, string, unicodedata                                                  # import text preprocessing libraries
import contractions                                                             # import contractions library
from bs4 import BeautifulSoup                                                   # import BeautifulSoup

from wordcloud import WordCloud, STOPWORDS, ImageColorGenerator                 # import wordcloud of a corpus, stopwords
import nltk                                                                     # import nltk library
from nltk.corpus import stopwords                                               # import nltk stopwords
from nltk.tokenize import word_tokenize, sent_tokenize                          # import Tokenizer

from sklearn.model_selection import train_test_split

# download stopwords
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to /Users/boyoon/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /Users/boyoon/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/boyoon/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [6]:
data = pd.read_csv("fake reviews dataset.csv")
data.head()

,category,rating,label,text_
0,Home_and_Kitchen_5,5.0,CG,"Love this! Well made, sturdy, and very comfor..."
1,Home_and_Kitchen_5,5.0,CG,"love it, a great upgrade from the original. I..."
2,Home_and_Kitchen_5,5.0,CG,This pillow saved my back. I love the look and...
3,Home_and_Kitchen_5,1.0,CG,"Missing information on how to use it, but it i..."
4,Home_and_Kitchen_5,5.0,CG,Very nice set. Good quality. We have had the s...


In [7]:
#remove the html tags
def strip_html(text):
    soup = BeautifulSoup(text, "html.parser")                    
    return soup.get_text()

#expand the contractions
def replace_contractions(text):
    """Replace contractions in string of text"""
    return contractions.fix(text)

#remove the numericals present in the text
def remove_numbers(text):
  text = re.sub(r'\d+', '', text)
  return text

def clean_text(text):
    text = strip_html(text)
    text = replace_contractions(text)
    text = remove_numbers(text)
    return text

data['text_'] = data['text_'].apply(lambda x: clean_text(x))
data.head()

/var/folders/r5/pzyqwyjx0dx2t_4tzqgy58c40000gn/T/ipykernel_2350/2670813475.py:3: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  soup = BeautifulSoup(text, "html.parser")


,category,rating,label,text_
0,Home_and_Kitchen_5,5.0,CG,"Love this! Well made, sturdy, and very comfor..."
1,Home_and_Kitchen_5,5.0,CG,"love it, a great upgrade from the original. I..."
2,Home_and_Kitchen_5,5.0,CG,This pillow saved my back. I love the look and...
3,Home_and_Kitchen_5,1.0,CG,"Missing information on how to use it, but it i..."
4,Home_and_Kitchen_5,5.0,CG,Very nice set. Good quality. We have had the s...


In [8]:
data['tokenized'] = data.apply(lambda row: nltk.word_tokenize(row['text_']), axis=1) # tokenization of data
data.head()

,category,rating,label,text_,tokenized
0,Home_and_Kitchen_5,5.0,CG,"Love this! Well made, sturdy, and very comfor...","[Love, this, !, Well, made, ,, sturdy, ,, and,..."
1,Home_and_Kitchen_5,5.0,CG,"love it, a great upgrade from the original. I...","[love, it, ,, a, great, upgrade, from, the, or..."
2,Home_and_Kitchen_5,5.0,CG,This pillow saved my back. I love the look and...,"[This, pillow, saved, my, back, ., I, love, th..."
3,Home_and_Kitchen_5,1.0,CG,"Missing information on how to use it, but it i...","[Missing, information, on, how, to, use, it, ,..."
4,Home_and_Kitchen_5,5.0,CG,Very nice set. Good quality. We have had the s...,"[Very, nice, set, ., Good, quality, ., We, hav..."


In [ ]:
stopwords = stopwords.words('english')
stopwords = list(set(stopwords)) + custom_remove_list

# convert all characters to lowercase
def to_lowercase(words):
    """Convert all characters to lowercase from list of tokenized words"""
    new_words = []
    for word in words:
        new_word = word.lower()
        new_words.append(new_word)
    return new_words

def remove_punctuation(words):
    """Remove punctuation from list of tokenized words"""
    new_words = []
    for word in words:
        new_word = re.sub(r'[^\w\s]', '', word)
        if new_word != '':
            new_words.append(new_word)
    return new_words

def normalize(words):
    words = to_lowercase(words)
    words = remove_punctuation(words)
    return ' '.join(words)

data['cleaned_text'] = data.apply(lambda row: normalize(row['tokenized']), axis=1)
data.head()

In [35]:
# Count punctuation
def count_punctuation(text):
    punctuation_count = sum(text.count(p) for p in string.punctuation)
    return punctuation_count

data['punctuation'] = data['text_'].apply(count_punctuation)
# Add count of number of words in review
data['words'] = data['tokenized'].apply(lambda x: len(x))

# Count number of characters in review (incl. punctuation but not spaces)

def count_non_space_chars(text):
    count = sum(1 for char in text if char != ' ')
    return count

# Create a new column 'char_count' to store the counts
data['characters'] = data['text_'].apply(count_non_space_chars)


In [36]:
def label_enc(text):
  return int(text == "CG")
data["label"] = data["label"].apply(lambda text: label_enc(text))
data

,category,rating,label,text_,tokenized,cleaned_text,punctuation,words,characters
0,Home_and_Kitchen_5,5.0,1,"Love this! Well made, sturdy, and very comfor...","[Love, this, !, Well, made, ,, sturdy, ,, and,...",love this well made sturdy and very comfortabl...,5,18,62
1,Home_and_Kitchen_5,5.0,1,"love it, a great upgrade from the original. I...","[love, it, ,, a, great, upgrade, from, the, or...",love it a great upgrade from the original i ha...,2,19,65
2,Home_and_Kitchen_5,5.0,1,This pillow saved my back. I love the look and...,"[This, pillow, saved, my, back, ., I, love, th...",this pillow saved my back i love the look and ...,2,16,54
3,Home_and_Kitchen_5,1.0,1,"Missing information on how to use it, but it i...","[Missing, information, on, how, to, use, it, ,...",missing information on how to use it but it is...,2,19,64
4,Home_and_Kitchen_5,5.0,1,Very nice set. Good quality. We have had the s...,"[Very, nice, set, ., Good, quality, ., We, hav...",very nice set good quality we have had the set...,2,20,68
...,...,...,...,...,...,...,...,...,...
40427,Clothing_Shoes_and_Jewelry_5,4.0,0,I had read some reviews saying that this bra r...,"[I, had, read, some, reviews, saying, that, th...",i had read some reviews saying that this bra r...,44,376,1347
40428,Clothing_Shoes_and_Jewelry_5,5.0,1,I was not sure exactly what it would be. It is...,"[I, was, not, sure, exactly, what, it, would, ...",i was not sure exactly what it would be it is ...,41,304,1002
40429,Clothing_Shoes_and_Jewelry_5,2.0,0,"You can wear the hood by itself, wear it with ...","[You, can, wear, the, hood, by, itself, ,, wea...",you can wear the hood by itself wear it with t...,84,411,1626
40430,Clothing_Shoes_and_Jewelry_5,1.0,1,I liked nothing about this dress. The only rea...,"[I, liked, nothing, about, this, dress, ., The...",i liked nothing about this dress the only reas...,40,319,997


In [37]:
# Add column that changes rating number to category
for index, row in data.iterrows():
    if row['rating'] == 5:
        data.at[index, 'rate'] = "high"
    elif row['rating'] == 4:
        data.at[index, 'rate'] = "med-high"
    elif row['rating'] == 3:
        data.at[index, 'rate'] = "medium"
    elif row['rating'] == 2:
        data.at[index, 'rate'] = "med-low"
    else:
        data.at[index, 'rate'] = "low" 

In [38]:
# Export to csv
data.to_csv("fake_review_cleaned.csv")